#  Android抓包与工具证书导入为系统证书.ipynb
　　参考：[]()  
　　　　　[]()  
　　　　　[抓包Android 7.0+将ca证书导入到系统（设置为系统证书），以及删除【用户凭据】证书](https://blog.csdn.net/tabactivity/article/details/113505112)  
　　　　　[]()  
　　　　　[]()  
　　　　　[]()  
　　　　　[]()  
　　　　　[]()  
　　　　　[]()  
 ## 一、Android的抓包工具
 　　Wireshark、Charles、Fiddler、Mitmproxy等。  
   
 ## 二、Android设置代理
 　　注意：Android抓包时，其WIFI网络必须与本地抓包工具的主机处于同一网段内！同时抓包工具还需开启HTTPS证书验证设置。 
 　　通过手机设置的网络设置，找到wifi网络链接选项，长按当前正在连接的wifi网络(5s以上)，在弹出框中选择修改网络，进而选高级选项，代理一栏选择手动，接下来设置代理服务器主机名为抓包主机的IP,端口为抓包工具监听端口即可。
 ## 三、Android系统证书导入
 　　注意：要想通过抓包工具实现手机等移动端抓包，必须将抓包工具的证书导入为android设备的系统证书，这就要求android设备必须具有root权限。  
 ### 3.1、导出工具证书
 #### 3.1.1、Fiddler证书导出
 　　打开Fiddler应用，点击右上角tools选项卡的options选项，在弹出框选择HTTPS选项，点击actions，进而点击Export Root Certificate to Desktop选项(配置好HTTPS证书验证支持),将Fiddler证书导出到本地桌面（文件：FiddlerRoot.cer）。
 #### 3.1.2、Charles证书导出
 

### 3.2、证书处理
　　注意：此处使用openssl工具（[安装链接](http://slproweb.com/products/Win32OpenSSL.html)，安装完成后，将安装目录的bin路径配置进环境变量path）。
#### 3.2.1、查看证书

>openssl x509 -in FiddlerRoot.crt -inform DER -noout -text　　　　# 查看crt\der格式证书  
openssl x509 -in certificate.pem -inform pem -noout -text　　　　# 查看pem证书   

#### 3.2.2、证书转换

>openssl x509 -in FiddlerRoot.crt -inform DER -out certificate.pem -outform PEM　　　　# crt/der转pem    
openssl x509 -in certificate.pem -inform PEM -out FiddlerRoot.der -outform DER　　　　# pem转der/crt  

### 3.3、证书导入为Android系统证书
#### 3.3.1、证书导入前准备
　　从抓包工具导出的证书一般为crt格式，需使用上面的证书转换命令将之转换为pem格式，并将pem格式文件重命名为形如c06587b6.0的格式。其中c06587b6为证书的哈希值，需使用如下命令获取（后面.0为防止文件名冲突所加,如果你的根证书文件夹里面已经有这个文件名了,那就将.0改为.1,以此类推）。  
>openssl x509 -subject_hash_old -in FiddlerRoot.pem　　　　# 命令输出的第一行形如c06587b6的字符串即为重命名字符串  
  
#### 3.3.2、证书导入到系统
```shell
adb root
adb remount
adb push c06587b6.0 /system/etc/security/cacerts/
adb shell
cd /system/etc/security/cacerts/
chmod 644 c06587b6.0
reboot    # 重启系统
```
　　设备自动重启后，在手机“设置-》安全和隐私-》加密与凭据-》信任的凭据-》系统” 一栏，可找到已安装的抓包工具证书，此时就可以实现android抓包了。
## 四、证书删除
　　Android手机的【安全和隐私】-》【加密与凭据】-》【用户凭据】 里上有很多垃圾、不用的证书文件，如何删除?     
　　1、尝试在“加密与凭据”页面，点击“清除凭据”删除。   
　　2、需要root权限方可删除，直接进目录删除（按需删除）：   
　　　　/data/misc/keystore/user_0/   